#  Modeling

This notebook will be used to create a model using generated test data.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../scripts/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import util

In [2]:
df = pd.read_csv('../data/features.csv', index_col=0)

In [3]:
target = 'Survived'

train = df.loc[pd.notnull(df['Survived'])]
test = df.loc[pd.isnull(df['Survived'])]

y = train[target]
labels = test['Id']

del train[target]
del test[target]
del train['Id']
del test['Id']

X = train.copy()

# Base Models

Train a number of different base models for our ensemble approach.

Create parameters. Taken from Anisotropic's [notebook](www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python)

In [4]:
seed = 1
# Random Forest
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'max_features': 'auto',
    'max_depth': 6,
    'min_samples_leaf': 2, 
    'random_state': seed
}

# Extra Trees
et_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'max_features': 'auto',
    'max_depth': 8,
    'min_samples_leaf': 2, 
    'random_state': seed
}

# AdaBoost
ada_params = {
    'n_estimators': 500,
    'learning_rate': 0.75,
    'random_state': seed
}

# Gradient Boosting
gb_params = {
    'n_estimators': 500,
    'max_depth': 5,
    'min_samples_leaf': 2, 
    'random_state': seed
}

# SVM
svc_params = {
    'kernel': 'linear',
    'C': 0.025
}

In [5]:
rf = ensemble.RandomForestClassifier(**rf_params)
et = ensemble.ExtraTreesClassifier(**et_params)
ada = ensemble.AdaBoostClassifier(**ada_params)
gb = ensemble.GradientBoostingClassifier(**gb_params)
svc = svm.SVC(**svc_params)

In [6]:
def print_acc(scores, name):
    print('Model ' + name + ' scored accuracy of %0.5f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

In [7]:
rf_scores = cross_val_score(rf, X, y, cv=10)
et_scores = cross_val_score(et, X, y, cv=10)
ada_scores = cross_val_score(ada, X, y, cv=10)
gb_scores = cross_val_score(gb, X, y, cv=10)
svc_scores = cross_val_score(svc, X, y, cv=10)

# Print results
print_acc(rf_scores, 'rf')
print_acc(et_scores, 'et')
print_acc(ada_scores, 'ada')
print_acc(gb_scores, 'gb')
print_acc(svc_scores, 'svc')

Model rf scored accuracy of 0.83282 (+/- 0.08)
Model et scored accuracy of 0.82830 (+/- 0.06)
Model ada scored accuracy of 0.82609 (+/- 0.06)
Model gb scored accuracy of 0.81714 (+/- 0.08)
Model svc scored accuracy of 0.80695 (+/- 0.06)


Create predictions on the test set.

In [8]:
base_test_predictions = pd.DataFrame()
base_test_predictions['rf'] = rf.fit(X, y).predict(test)
base_test_predictions['et'] = et.fit(X, y).predict(test)
base_test_predictions['ada'] = ada.fit(X, y).predict(test)
base_test_predictions['gb'] = gb.fit(X, y).predict(test)
base_test_predictions['svc'] = svc.fit(X, y).predict(test)

## Feature Importance

# Second Layer

In [10]:
util.save_submission(base_test_predictions.mode(axis=1)[0].ravel(), labels, name='../submissions/majority.csv') 